# 8. Using rqt_console and roslaunch
http://wiki.ros.org/ROS/Tutorials/UsingRqtconsoleRoslaunch

In [1]:
import os
import sys

home_dir = os.getcwd()
home_dir

'/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials'

In [2]:
# get ROS related environment variables
envs = !printenv | grep ROS 
ROS_ENVs = {}

for env in envs:
    env = env.split('=')
    [k, v] = env
    ROS_ENVs[k] = v

ROS_ENVs['LD_LIBRARY_PATH'] = '/opt/ros/kinetic/lib'
ROS_ENVs['PKG_CONFIG_PATH'] = '/opt/ros/kinetic/lib/pkgconfig'
ROS_ENVs['CMAKE_PREFIX_PATH'] = '/opt/ros/kinetic'  
del ROS_ENVs['PWD']

## Setup
### roscore
Let's start by making sure that we have roscore running:  

open a new terminal window, run a ROS Docker container with:
```
docker run -it --rm \
	--net=host \
	-v /tmp/.X11-unix:/tmp/.X11-unix \
	-e DISPLAY=unix$DISPLAY \
	--cap-add=SYS_ADMIN \
	--device /dev/snd \
	--device /dev/dri \
	-v /dev/shm:/dev/shm \
	-p 11311:11311 \
	wei1234c/ros_ubuntu_amd64
```

in the container, run roscore:
```
$ roscore
```

### turtlesim
For this tutorial we will also use turtlesim.  
open a new terminal window, run a ROS Docker container with:
```
docker run -it --rm \
	--net=host \
	-v /tmp/.X11-unix:/tmp/.X11-unix \
	-e DISPLAY=unix$DISPLAY \
	--cap-add=SYS_ADMIN \
	--device /dev/snd \
	--device /dev/dri \
	-v /dev/shm:/dev/shm \
	-p 11311:11311 \
	wei1234c/ros_ubuntu_amd64
```

in the container, run:
```
$ rosrun turtlesim turtlesim_node
```

### turtle keyboard teleoperation
We'll also need something to drive the turtle around with.  

open a new terminal window, run a ROS Docker container with:
```
docker run -it --rm \
	--net=host \
	-v /tmp/.X11-unix:/tmp/.X11-unix \
	-e DISPLAY=unix$DISPLAY \
	--cap-add=SYS_ADMIN \
	--device /dev/snd \
	--device /dev/dri \
	-v /dev/shm:/dev/shm \
	-p 11311:11311 \
	wei1234c/ros_ubuntu_amd64
```

in the container, run:
```
$ rosrun turtlesim turtle_teleop_key
```

Now you can use the arrow keys of the keyboard to drive the turtle around. If you can not drive the turtle, **select the terminal window of the turtle_teleop_key** to make sure that the keys that you type are recorded.

## Prerequisites rqt and turtlesim package
The tutorial uses both the rqt and turtlesim packages. To do this tutorial, please install both packages, if you have not yet done so.
```
$ sudo apt-get install ros-kinetic-rqt ros-kinetic-rqt-common-plugins ros-kinetic-turtlesim
``` 

### Using rqt_console and rqt_logger_level  
rqt_console attaches to ROS's logging framework to display output from nodes. rqt_logger_level allows us to change the verbosity level (DEBUG, WARN, INFO, and ERROR) of nodes as they run.

Now let's look at the turtlesim output in **rqt_console** and switch logger levels in **rqt_logger_level** as we use turtlesim. Before we start the turtlesim, in two new terminals start rqt_console and rqt_logger_level:

### run rqt_console
open a new terminal window, run a ROS Docker container with:
```
docker run -it --rm \
	--net=host \
	-v /tmp/.X11-unix:/tmp/.X11-unix \
	-e DISPLAY=unix$DISPLAY \
	--cap-add=SYS_ADMIN \
	--device /dev/snd \
	--device /dev/dri \
	-v /dev/shm:/dev/shm \
	-p 11311:11311 \
	wei1234c/ros_ubuntu_amd64
```

in the container, run:
```
$ rosrun rqt_console rqt_console
```  

<img src='http://wiki.ros.org/ROS/Tutorials/UsingRqtconsoleRoslaunch?action=AttachFile&do=get&target=rqt_console%28start%29.png'>

### run rqt_logger_level

open a new terminal window, run a ROS Docker container with:
```
docker run -it --rm \
	--net=host \
	-v /tmp/.X11-unix:/tmp/.X11-unix \
	-e DISPLAY=unix$DISPLAY \
	--cap-add=SYS_ADMIN \
	--device /dev/snd \
	--device /dev/dri \
	-v /dev/shm:/dev/shm \
	-p 11311:11311 \
	wei1234c/ros_ubuntu_amd64
```

in the container, run:
```
$ rosrun rqt_logger_level rqt_logger_level
```    

<img src='http://wiki.ros.org/ROS/Tutorials/UsingRqtconsoleRoslaunch?action=AttachFile&do=get&target=rqt_logger_level.png'>

Now let's change the logger level to Warn by refreshing the nodes in the rqt_logger_level window and selecting Warn as shown below:
<img src='http://wiki.ros.org/ROS/Tutorials/UsingRqtconsoleRoslaunch?action=AttachFile&do=get&target=rqt_logger_level%28error%29.png'>

Now let's run our turtle into the wall and see what is displayed in our rqt_console:

In [3]:
!rostopic pub /turtle1/cmd_vel geometry_msgs/Twist -r 1 -- '{linear: {x: 2.0, y: 0.0, z: 0.0}, angular: {x: 0.0,y: 0.0,z: 0.0}}'

^C


<img src='http://wiki.ros.org/ROS/Tutorials/UsingRqtconsoleRoslaunch?action=AttachFile&do=get&target=rqt_console%28turtlesimerror%29.png'>

### Quick Note about logger levels
Logging levels are prioritized in the following order:
```
Fatal
Error
Warn
Info
Debug
```

Fatal has the highest priority and Debug has the lowest. By setting the logger level, you will get all messages of that priority level or higher. For example, by setting the level to Warn, you will get all Warn, Error, and Fatal logging messages.

### Using roslaunch
roslaunch starts nodes as defined in a launch file.

Usage:
```
$ roslaunch [package] [filename.launch]
```
First go to the beginner_tutorials package we created and built earlier:




In [3]:
import os

# define workspace path
ws_dir = 'catkin_ws'
ws_dir_abs = os.path.join(home_dir, ws_dir)
src_dir_abs = os.path.join(ws_dir_abs, 'src') 

# append environment paths
def gen_paths(ws_dir_abs, src_dir_abs):
    paths = {}
    paths['ROS_PACKAGE_PATH'] = src_dir_abs
    paths['LD_LIBRARY_PATH'] = os.path.join(ws_dir_abs + 'devel/lib')
    paths['ROSLISP_PACKAGE_DIRECTORIES'] = os.path.join(ws_dir_abs + 'devel/share/common-lisp')
    paths['PKG_CONFIG_PATH'] = os.path.join(ws_dir_abs + 'devel/lib/pkgconfig')
    paths['CMAKE_PREFIX_PATH'] = os.path.join(ws_dir_abs + 'devel/lib')
    paths['LD_LIBRARY_PATH'] = os.path.join(ws_dir_abs + 'devel')
    return paths

def append_paths(ROS_ENVs, paths): 
    for path in ROS_ENVs.keys():
        os.environ[path] = ROS_ENVs[path]
        if path in paths.keys():
            os.environ[path] = '{}:{}'.format(paths[path], os.environ[path])

append_paths(ROS_ENVs, 
             gen_paths(ws_dir_abs, src_dir_abs))

In [5]:
# find package path
package_name = 'beginner_tutorials'

package_dir_abs = !rospack find {package_name}
package_dir_abs = package_dir_abs[0]
package_dir_abs

'/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/catkin_ws/src/beginner_tutorials'

In [6]:
cd {package_dir_abs}

/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/catkin_ws/src/beginner_tutorials


make a launch directory:

In [7]:
mkdir launch

In [8]:
cd launch

/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/catkin_ws/src/beginner_tutorials/launch


create a launch file called **turtlemimic.launch** and paste the following:

In [9]:
text = '''
<launch>

  <group ns="turtlesim1">
    <node pkg="turtlesim" name="sim" type="turtlesim_node"/>
  </group>

  <group ns="turtlesim2">
    <node pkg="turtlesim" name="sim" type="turtlesim_node"/>
  </group>

  <node pkg="turtlesim" name="mimic" type="mimic">
    <remap from="input" to="turtlesim1/turtle1"/>
    <remap from="output" to="turtlesim2/turtle1"/>
  </node>

</launch>
'''

file_name = 'turtlemimic.launch'

with open(file_name, 'w') as f:
    f.writelines(text)

Here we start two groups with a namespace tag of turtlesim1 and turtlesim2 with a turtlesim node with a name of sim. This allows us to start two simulators without having name conflicts.
```
  <group ns="turtlesim1">
    <node pkg="turtlesim" name="sim" type="turtlesim_node"/>
  </group>

  <group ns="turtlesim2">
    <node pkg="turtlesim" name="sim" type="turtlesim_node"/>
  </group>
```

Here we start the mimic node with the topics input and output renamed to turtlesim1 and turtlesim2. This renaming will cause turtlesim2 to mimic turtlesim1.
```
  <node pkg="turtlesim" name="mimic" type="mimic">
    <remap from="input" to="turtlesim1/turtle1"/>
    <remap from="output" to="turtlesim2/turtle1"/>
  </node>
```

### roslaunching
Now let's roslaunch the launch file:

In [ ]:
!roslaunch {package_name} turtlemimic.launch

... logging to /home/pi/.ros/log/a01a9306-233f-11e8-b375-0242d121c5a4/roslaunch-wei-Lenovo-Legion-1580.log
Checking log directory for disk usage. This may take awhile.
Press Ctrl-C to interrupt
Done checking log file disk usage. Usage is <1GB.
]2;/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/catkin_ws/src/beginner_tutorials/launch/turtlemimic.launch
started roslaunch server http://wei-Lenovo-Legion:34065/

SUMMARY

PARAMETERS
 * /rosdistro: kinetic
 * /rosversion: 1.12.12

NODES
  /
    mimic (turtlesim/mimic)
  /turtlesim1/
    sim (turtlesim/turtlesim_node)
  /turtlesim2/
    sim (turtlesim/turtlesim_node)

ROS_MASTER_URI=http://localhost:11311
]2;/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials/catkin_ws/src/beginner_tutorials/launch/turtlemimic.launch http://localhost:11311
process[turtlesim1/sim-1]: started with pid [1589]
libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast
Could not initialize OpenGL for RasterGLSurface, reverti

Two turtlesims will start and in a new terminal send the rostopic command:

For ROS Hydro and later,

open a new terminal window, run a ROS Docker container with:
```
docker run -it --rm \
	--net=host \
	-v /tmp/.X11-unix:/tmp/.X11-unix \
	-e DISPLAY=unix$DISPLAY \
	--cap-add=SYS_ADMIN \
	--device /dev/snd \
	--device /dev/dri \
	-v /dev/shm:/dev/shm \
	-p 11311:11311 \
	wei1234c/ros_ubuntu_amd64
```

in the container, run:
```
$rostopic pub /turtlesim1/turtle1/cmd_vel geometry_msgs/Twist -r 1 -- '[2.0, 0.0, 0.0]' '[0.0, 0.0, -1.8]'
```

We can also use rqt_graph to better understand what our launch file did. Run rqt's main window and select Plugins > Introspection > Node Graph:

```
$ rqt
``` 

Or simply:

``` 
$ rqt_graph
```

<img src='http://wiki.ros.org/ROS/Tutorials/UsingRqtconsoleRoslaunch?action=AttachFile&do=get&target=mimiclaunch.jpg'>